<a href="https://colab.research.google.com/github/adiManethia/Twitter-Sentiment-Analysis-using-ML/blob/main/TwitterSentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/adiManethia/Twitter-Sentiment-Analysis-using-ML

fatal: destination path 'Twitter-Sentiment-Analysis-using-ML' already exists and is not an empty directory.


In [2]:
# installing kaggle library
! pip install kaggle

## upload kaggle.json file

In [3]:
# configuring the path of kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

### Import twitter sentiment dataset using kaggle api

In [4]:
# API to fetch the dataset from kaggle
!kaggle datasets download -d kazanova/sentiment140

sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [5]:
# extracting the compressed datasets

from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print('The dataset is extraxted')

The dataset is extraxted


### Importing the Dependencies

In [6]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
# print stopwords in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

### Data Processing

In [9]:
# load the data from csv file to pandas dataframe
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1')  # this iso is very commonn for encoding

In [10]:
# check number of rows and columns
twitter_data.shape

(1599999, 6)

In [11]:
twitter_data.head()  # first 5 rows

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


Here, we can see that first tweet is being read as column name, we have to change it.

In [12]:
# name the columns and read the dataset again

column_names= ['target', 'id','date','flag','user','text']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', names = column_names, encoding='ISO-8859-1')

In [13]:
# check shape again
twitter_data.shape

(1600000, 6)

In [14]:
# first 5 rows
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [15]:
# check if there are any missing values/text (count them)
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

##### so, we can no values are missing

In [16]:
# checking the distribution of target colums
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [17]:
# labels are not in coreect fomat, let's change 4 into 1
twitter_data.replace({'target':{4:1}}, inplace=True)


In [18]:
twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

0------> Negative Tweet
1------> Positive Tweet

## Stemming

stemming is a process of reducing a word to it's root word.

In [19]:
port_stem = PorterStemmer()


In [20]:
def stemming(content):

  stemmed_content = re.sub('[^a-zA-Z]','',content)  # remove all the letters that are not alphabets, re is regular expression with sub module
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ''.join(stemmed_content)

  return stemmed_content

In [21]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)  # 5 mins to complete execution

In [22]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoothttptwitpiccomyzlawwwthatsabummeryou...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,isupsetthathecantupdatehisfacebookbytextingita...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichanidivedmanytimesfortheballmanagedtosave...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,mywholebodyfeelsitchyandlikeitsonfir
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclassnoitsnotbehavingatallimmadwhyam...


In [23]:
## we will use target and stemmed content columns to train our model


In [24]:
print(twitter_data['stemmed_content'])

0          switchfoothttptwitpiccomyzlawwwthatsabummeryou...
1          isupsetthathecantupdatehisfacebookbytextingita...
2          kenichanidivedmanytimesfortheballmanagedtosave...
3                       mywholebodyfeelsitchyandlikeitsonfir
4          nationwideclassnoitsnotbehavingatallimmadwhyam...
                                 ...                        
1599995           justwokeuphavingnoschoolisthebestfeelingev
1599996    thewdbcomverycooltohearoldwaltinterviewshttpbl...
1599997         areyoureadyforyourmojomakeoveraskmefordetail
1599998     happythbirthdaytomybooofallltimetupacamarushakur
1599999    happycharitytuesdaythenspccsparkscharityspeaki...
Name: stemmed_content, Length: 1600000, dtype: object


In [25]:
# separating the data and label
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

In [26]:
print(X)

['switchfoothttptwitpiccomyzlawwwthatsabummeryoushouldagotdavidcarrofthirddaytodoitd'
 'isupsetthathecantupdatehisfacebookbytextingitandmightcryasaresultschooltodayalsoblah'
 'kenichanidivedmanytimesfortheballmanagedtosavetherestgooutofbound' ...
 'areyoureadyforyourmojomakeoveraskmefordetail'
 'happythbirthdaytomybooofallltimetupacamarushakur'
 'happycharitytuesdaythenspccsparkscharityspeakinguphh']


In [27]:
print(Y)

[0 0 0 ... 1 1 1]


## Spliting the data in training anf test data

In [28]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, stratify=Y, random_state=42) # stratify will ensure good distribution of 0 and 1 in train and test data

In [29]:
print(X.shape, X_train.shape, X_test.shape)

(1600000,) (1280000,) (320000,)


In [30]:
print(X_train)
print(X_test)

['paisleypaisleylolwhydoigetideassofarinadvanceitsnotevenjuneyetweneedathirdknittertohaveourownsummergroup'
 'worstheadacheev'
 'ewaniesciuszkoiamsosadiwontseeyouimissyoualreadyandyeahthatsperfecticomebacktheth'
 ...
 'itsamandijustgothomefrommeetingandtalkingendlesslywithoneofthecoolestguysshesevermetsmil'
 'justboughtchocolatebarswithquotinorwinsafreebarquotlabelsididntwinoneith'
 'miseciaallisaidisthatihopeitdoescandmoremailmesunday']
['stmdenalieyesblackandredaremyfavcolorsireallywantedthemandthosecolorsdeflooksawesomeonjar'
 'qubuyopenhousethisweekendampmbestvalueonebedroominliclongislandcitybdhttptinyurlcomptnqd'
 'ginoandfranfrancanyougreetmeonairifthatsokayhahahahathank' ...
 'labratfollowingyouinherealsoihopeyouwillatleastgetthisalsowishesforgettingwellsoon'
 'ifeellikewehaventhadadecentswellsincelastfallandnohopeofwavesformyrtlebeachthisweekeitheratleastthereisgolf'
 'relaxinafterabusyday']


In [31]:
## Ml model does not understand text, we have to convert text into vectors

In [32]:
# convert text into numerical data
vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)  # fit means it will try to understand the importance of words while converting it to numerical data
X_test = vectorizer.transform(X_test)  # do not use fit here as we have to test it eventually

In [33]:
print(X_train)


  (0, 885717)	1.0
  (1, 1224936)	1.0
  (2, 262037)	1.0
  (3, 221908)	1.0
  (4, 922157)	1.0
  (5, 858419)	1.0
  (6, 970110)	1.0
  (7, 910824)	1.0
  (8, 969541)	1.0
  (9, 225424)	1.0
  (10, 421451)	1.0
  (11, 497301)	1.0
  (12, 930254)	1.0
  (13, 705712)	1.0
  (14, 9777)	1.0
  (15, 736777)	1.0
  (16, 6424)	1.0
  (17, 209828)	1.0
  (18, 59252)	1.0
  (19, 1083962)	1.0
  (20, 1077830)	1.0
  (21, 815711)	1.0
  (22, 718567)	1.0
  (23, 625442)	1.0
  (24, 402401)	1.0
  :	:
  (1279975, 1093591)	1.0
  (1279976, 119267)	1.0
  (1279977, 32581)	1.0
  (1279978, 38427)	1.0
  (1279979, 829366)	1.0
  (1279980, 529361)	1.0
  (1279981, 849096)	1.0
  (1279982, 298687)	1.0
  (1279983, 879049)	1.0
  (1279984, 601472)	1.0
  (1279985, 739087)	1.0
  (1279986, 1232621)	1.0
  (1279987, 368514)	1.0
  (1279988, 1131384)	1.0
  (1279989, 1112441)	1.0
  (1279990, 1157566)	1.0
  (1279991, 1206993)	1.0
  (1279992, 56903)	1.0
  (1279993, 1056546)	1.0
  (1279994, 383300)	1.0
  (1279995, 818434)	1.0
  (1279996, 867177)	1.0

In [34]:
print(X_test)

  (26, 363104)	1.0
  (86, 363319)	1.0
  (206, 815419)	1.0
  (225, 926130)	1.0
  (264, 926139)	1.0
  (310, 806049)	1.0
  (318, 382976)	1.0
  (367, 377938)	1.0
  (368, 270456)	1.0
  (370, 324286)	1.0
  (374, 532173)	1.0
  (394, 45193)	1.0
  (437, 14042)	1.0
  (500, 363725)	1.0
  (600, 760437)	1.0
  (688, 452443)	1.0
  (772, 537175)	1.0
  (839, 322768)	1.0
  (846, 625831)	1.0
  (849, 993492)	1.0
  (869, 520084)	1.0
  (905, 308237)	1.0
  (957, 539617)	1.0
  (1101, 79547)	1.0
  (1176, 481611)	1.0
  :	:
  (318932, 67757)	1.0
  (319124, 1244132)	1.0
  (319302, 329000)	1.0
  (319350, 754979)	1.0
  (319394, 737013)	1.0
  (319395, 319205)	1.0
  (319421, 374565)	1.0
  (319466, 363336)	1.0
  (319541, 1228699)	1.0
  (319602, 164254)	1.0
  (319612, 134901)	1.0
  (319716, 776389)	1.0
  (319756, 1137315)	1.0
  (319765, 824809)	1.0
  (319803, 381801)	1.0
  (319804, 554064)	1.0
  (319807, 228521)	1.0
  (319824, 754116)	1.0
  (319855, 1238754)	1.0
  (319883, 513191)	1.0
  (319929, 115043)	1.0
  (319948, 

Training the machine learning Model

In [35]:
## Here, we are using Logistic regression as it is a classification problem

In [36]:
# set the penalty as regularization type l1/l2/elastic net to overcome overfitting
penalty = 'l2'
C = 0.01 # regularizatio strength

In [37]:
model = LogisticRegression(max_iter=100, penalty=penalty, C=C)

In [38]:
model.fit(X_train, Y_train)

LogisticRegression(C=0.01)

In [39]:
# model trys to understand what are the words that corresponds to negative tweet (0) and positive tweets (1)

### Model evaluation

Accuracy score

In [40]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction) # true values and predected values

In [41]:
print('Accuracy score for training data: ', training_data_accuracy)

Accuracy score for training data:  0.99811875


In [42]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)

In [43]:
print('Accuracy score for test data: ', test_data_accuracy)

Accuracy score for test data:  0.512203125


In [49]:
## save the model

In [50]:
import pickle

In [52]:
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))